In [2]:
# ---------------- DB CONNECTION ----------------

import mysql.connector

# Connect to MySQL (no database selected yet)
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="12345",
    database="cricbuzzz_db"
)

cursor = conn.cursor()

# Create the database
cursor.execute("CREATE DATABASE IF NOT EXISTS cricbuzzz_db")
print("Database cricbuzzz_db created (if it didn't already exist)")



Database cricbuzzz_db created (if it didn't already exist)


In [9]:
#adding the match table

cursor.execute("""
    CREATE TABLE IF NOT EXISTS matches (
    match_id BIGINT PRIMARY KEY,
    series_id BIGINT,
    series_name VARCHAR(255),
    match_desc VARCHAR(255),
    match_format VARCHAR(50),
    match_type VARCHAR(50),
    start_date BIGINT,
    end_date BIGINT,
    state VARCHAR(50),
    status VARCHAR(255),
    team1 VARCHAR(255),
    team2 VARCHAR(255),
    venue VARCHAR(255)
)
""")

print("✅ Matches table created")


conn.commit()
# cursor.close()
# conn.close()

✅ Matches table created


In [116]:
#Adding the palyers table

cursor.execute("""
    CREATE TABLE IF NOT EXISTS players (
    player_id BIGINT PRIMARY KEY,
    name VARCHAR(255),
    team VARCHAR(255),
    role VARCHAR(50),
    batting_style VARCHAR(50),
    bowling_style VARCHAR(50),
    dob VARCHAR(50),
    country VARCHAR(100)
    )
""")

print("✅ Players Table created")
conn.commit()

✅ Players Table created


In [117]:
#Adding match Table

cursor.execute("""
    CREATE TABLE IF NOT EXISTS match_innings (
    id INT AUTO_INCREMENT PRIMARY KEY,
    match_id BIGINT,
    team_id BIGINT,
    team_name VARCHAR(255),
    runs INT,
    wickets INT,
    overs VARCHAR(10),
    FOREIGN KEY (match_id) REFERENCES matches(match_id)
    )
    """)

print("✅  match_innings Table created")
conn.commit()


✅  match_innings Table created


In [118]:
#Adding Career Stats

cursor.execute("""
    CREATE TABLE IF NOT EXISTS batting_stats (
    id INT AUTO_INCREMENT PRIMARY KEY,
    player_id BIGINT,
    format VARCHAR(50),         
    matches INT,
    innings INT,
    runs INT,
    balls INT,
    highest_score VARCHAR(10),
    average FLOAT,
    strike_rate FLOAT,
    hundreds INT,
    fifties INT,
    fours INT,
    sixes INT,
    FOREIGN KEY (player_id) REFERENCES players(player_id)
)
""")

print("✅  batting_stats Table created")
conn.commit()


✅  batting_stats Table created


In [119]:
#Bowling carre stats

cursor.execute("""
    CREATE TABLE IF NOT EXISTS bowling_stats (
    id INT AUTO_INCREMENT PRIMARY KEY,
    player_id BIGINT,
    format VARCHAR(50),         
    matches INT,
    innings INT,
    balls INT,
    runs INT,
    wickets INT,
    best_figures VARCHAR(10),
    average FLOAT,
    economy FLOAT,
    strike_rate FLOAT,
    five_wickets INT,
    FOREIGN KEY (player_id) REFERENCES players(player_id)
)
""")

print("✅  bowling_stats Table created")
conn.commit()

✅  bowling_stats Table created


In [120]:
#batting scorecard

cursor.execute("""
    CREATE TABLE IF NOT EXISTS batting_scorecard (
    id INT AUTO_INCREMENT PRIMARY KEY,
    match_id BIGINT,
    innings_no INT,
    player_id BIGINT,
    player_name VARCHAR(255),
    runs INT,
    balls INT,
    fours INT,
    sixes INT,
    strike_rate FLOAT,
    dismissal VARCHAR(255),
    FOREIGN KEY (match_id) REFERENCES matches(match_id),
    FOREIGN KEY (player_id) REFERENCES players(player_id)
)
""")

print("✅  batting_scorecard Table created")
conn.commit()

✅  batting_scorecard Table created


In [121]:
#Bowling scorecard

cursor.execute("""
    CREATE TABLE IF NOT EXISTS bowling_scorecard (
    id INT AUTO_INCREMENT PRIMARY KEY,
    match_id BIGINT,
    innings_no INT,
    player_id BIGINT,
    player_name VARCHAR(255),
    overs VARCHAR(10),
    maidens INT,
    runs_conceded INT,
    wickets INT,
    economy FLOAT,
    FOREIGN KEY (match_id) REFERENCES matches(match_id),
    FOREIGN KEY (player_id) REFERENCES players(player_id)
)
""")

print("✅  bowling_scorecard Table created")
conn.commit()

✅  bowling_scorecard Table created


In [136]:
#Adding series Table

# cursor.execute("""
#     CREATE TABLE IF NOT EXISTS series (
#     series_id BIGINT PRIMARY KEY,
#     name VARCHAR(255),
#     start_date BIGINT,
#     end_date BIGINT,
#     series_type VARCHAR(50),
#     status VARCHAR(50)
# )
# """)

# print("✅  Series Table created")
# conn.commit()


# 1. Drop old table (if exists)
cursor.execute("DROP TABLE IF EXISTS series;")

# 2. Create new table with only 4 columns
cursor.execute("""
    CREATE TABLE series (
        series_id BIGINT PRIMARY KEY,
        name VARCHAR(255),
        start_date BIGINT,
        end_date BIGINT
    )
""")

conn.commit()
print("✅ Series table recreated successfully")


✅ Series table recreated successfully


In [123]:
#Adding match_players

cursor.execute("""
    CREATE TABLE IF NOT EXISTS match_players (
    id INT AUTO_INCREMENT PRIMARY KEY,
    match_id BIGINT,
    player_id BIGINT,
    team_name VARCHAR(255),
    role VARCHAR(50),
    FOREIGN KEY (match_id) REFERENCES matches(match_id),
    FOREIGN KEY (player_id) REFERENCES players(player_id)
)
""")

print("✅  Match_Players Table created")
conn.commit()

✅  Match_Players Table created


In [16]:
#Adding ICC Ranking Table

cursor.execute("""
    CREATE TABLE IF NOT EXISTS icc_rankings (
    id INT AUTO_INCREMENT PRIMARY KEY,
         
    player_id BIGINT,
    player_name VARCHAR(255),
    country VARCHAR(50),
    `rank` INT,
    rating INT,
    FOREIGN KEY (player_id) REFERENCES players(player_id)
)
""")

print("✅  ICC Ranking Table created")
conn.commit()

✅  ICC Ranking Table created


# ---------------- FETCH FUNCTIONS ----------------

# Insert series

In [143]:
import requests

BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"
API_KEY = "04554b6332msha8e0ac7707a698dp141329jsn4ddcfe7e1626"

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}



#Insert series
from datetime import datetime

def insert_series(series):
    # Keep BIGINT (raw millis) instead of converting to date string
    start_dt = int(series.get("startDt")) if series.get("startDt") else None
    end_dt   = int(series.get("endDt")) if series.get("endDt") else None

    cursor.execute("""
        INSERT INTO series (series_id, name, start_date, end_date)
        VALUES (%s,%s,%s,%s)
        ON DUPLICATE KEY UPDATE
          name = VALUES(name),
          start_date = VALUES(start_date),
          end_date = VALUES(end_date);
    """, (
        series.get("id"),
        series.get("name"),
        start_dt,
        end_dt
    ))
    print(f"✅ Series Inserted: {series.get('name')}")
    conn.commit()


    

# ---------------------- Series ----------------------
def fetch_series():
    url = "https://cricbuzz-cricket.p.rapidapi.com/series/v1/international"
    response = requests.get(url, headers=headers)
    data = response.json()
    return data.get("seriesMapProto", [])

print("Fetching Series...")
series_data = fetch_series()
print(series_data)

for s in series_data:  # this is a wrapper
    for ser in s.get("series", []):   # loop actual series
        insert_series(ser)
    print("✅ Data Inserting")



Fetching Series...
[{'date': 'FEBRUARY 2024', 'series': [{'id': 7572, 'name': 'ICC Cricket World Cup League Two 2023-27', 'startDt': '1707955200000', 'endDt': '1806364800000'}]}, {'date': 'AUGUST 2025', 'series': [{'id': 9602, 'name': 'South Africa tour of Australia, 2025', 'startDt': '1754784000000', 'endDt': '1755993600000'}, {'id': 9686, 'name': 'India tour of Bangladesh, 2025 (Called off)', 'startDt': '1755388800000', 'endDt': '1756598400000'}, {'id': 10719, 'name': 'Czech Republic tour of Romania 2025', 'startDt': '1755734400000', 'endDt': '1755820800000'}, {'id': 10746, 'name': 'Belgium tour of Austria 2025', 'startDt': '1755907200000', 'endDt': '1755993600000'}, {'id': 10790, 'name': 'Eastern Europe Cup 2025', 'startDt': '1756425600000', 'endDt': '1756598400000'}, {'id': 10642, 'name': 'United Arab Emirates T20I Tri-Series 2025', 'startDt': '1756425600000', 'endDt': '1757203200000'}, {'id': 10267, 'name': 'Sri Lanka tour of Zimbabwe, 2025', 'startDt': '1756425600000', 'endDt': '

#  ICC Rankings 


In [36]:
def insert_icc_ranking(rank):
    cursor.execute("""
        INSERT INTO icc_rankings ( player_id, player_name, country, `rank`,  rating)
        VALUES (%s,%s,%s,%s,%s)
    """, (
        rank.get("id"),
        rank.get("name"),
        rank.get("country"),
        rank.get("rank"),
        rank.get("rating")
        
    ))
    print("✅ Data Inserted")


    conn.commit()



import requests

BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"
API_KEY = "9d19b813acmsh770d33e52e3a171p11379djsn7dd0eed30400"

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

def fetch_icc_rankings():
    url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/rankings/batsmen"
    querystring = {"formatType":"test"}
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    return data

print("Fetching ICC Rankings...")

icc_data = fetch_icc_rankings()
print(icc_data)
for r in icc_data.get('rank',[]):
    insert_icc_ranking(r)
    print("✅ Data Inserting")

Fetching ICC Rankings...
{'rank': [{'id': '8019', 'rank': '1', 'name': 'Joe Root', 'country': 'England', 'rating': '908', 'points': '908', 'lastUpdatedOn': '2025-08-23', 'trend': 'Flat', 'faceImageId': '717792', 'countryId': '9'}, {'id': '12201', 'rank': '2', 'name': 'Harry Brook', 'country': 'England', 'rating': '868', 'difference': 1, 'points': '868', 'lastUpdatedOn': '2025-08-23', 'trend': 'Up', 'faceImageId': '717793', 'countryId': '9'}, {'id': '6326', 'rank': '3', 'name': 'Kane Williamson', 'country': 'New Zealand', 'rating': '850', 'difference': -1, 'points': '850', 'lastUpdatedOn': '2025-08-23', 'trend': 'Down', 'faceImageId': '616418', 'countryId': '13'}, {'id': '2250', 'rank': '4', 'name': 'Steven Smith', 'country': 'Australia', 'rating': '816', 'points': '816', 'lastUpdatedOn': '2025-08-23', 'trend': 'Flat', 'faceImageId': '619873', 'countryId': '4'}, {'id': '13940', 'rank': '5', 'name': 'Yashasvi Jaiswal', 'country': 'India', 'rating': '792', 'points': '792', 'lastUpdatedOn'

# Matches

In [7]:
import requests


BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"
API_KEY = "a71f9924b4msh7b2e6185f1bc0a7p1a92d3jsn353ec993d3ca"

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

# ----------- INSERT FUNCTION -------------

from datetime import datetime

# Helper to convert epoch milliseconds to datetime
def epoch_ms_to_datetime(epoch_ms):
    if not epoch_ms:
        return None
    return datetime.fromtimestamp(int(epoch_ms) / 1000)

def insert_match(match):
    info = match.get("matchInfo", {})
    score = match.get("matchScore", {})

    team1_score = score.get("team1Score", {}).get("inngs1", {})
    team2_score = score.get("team2Score", {}).get("inngs1", {})

    # Convert start/end dates to Python datetime
    start_dt = epoch_ms_to_datetime(info.get("startDate"))
    end_dt = epoch_ms_to_datetime(info.get("endDate"))

    cursor.execute("""
        INSERT INTO matches (
            match_id, series_id, series_name, match_desc, match_format, match_type,
            start_date_new, end_date_new, state, status, team1, team2, venue,
            team1_runs, team1_wkts, team1_overs,
            team2_runs, team2_wkts, team2_overs
        )
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        ON DUPLICATE KEY UPDATE 
            status=VALUES(status),
            team1_runs=VALUES(team1_runs),
            team1_wkts=VALUES(team1_wkts),
            team1_overs=VALUES(team1_overs),
            team2_runs=VALUES(team2_runs),
            team2_wkts=VALUES(team2_wkts),
            team2_overs=VALUES(team2_overs);
    """, (
        info.get("matchId"),
        info.get("seriesId"),
        info.get("seriesName"),
        info.get("matchDesc"),
        info.get("matchFormat"),
        info.get("matchType"),
        start_dt,          # ✅ converted
        end_dt,            # ✅ converted
        info.get("state"),
        info.get("status"),
        info.get("team1", {}).get("teamName"),
        info.get("team2", {}).get("teamName"),
        info.get("venueInfo", {}).get("ground"),
        team1_score.get("runs"),
        team1_score.get("wickets"),
        team1_score.get("overs"),
        team2_score.get("runs"),
        team2_score.get("wickets"),
        team2_score.get("overs")
    ))
    conn.commit()
    print(f"✅ Match {info.get('matchId')} inserted/updated")



# ----------- FETCH MATCHES -------------
def fetch_matches(status="all"):
    if status == "all":
        url = f"{BASE_URL}/matches/v1"
    elif status in ["live", "upcoming", "recent"]:
        url = f"{BASE_URL}/matches/v1/{status}"
    else:
        raise ValueError("Invalid status, must be all/live/upcoming/recent")

    response = requests.get(url, headers=headers)
    data = response.json()

    matches = []
    for type_match in data.get("typeMatches", []):
        for series in type_match.get("seriesMatches", []):
            series_data = series.get("seriesAdWrapper", {})
            for match in series_data.get("matches", []):
                matches.append(match)   # 👈 keep full object (info + score)
    return matches


# ----------- RUNNING -------------
print("Fetching Upcoming Matches...")
upcoming_matches = fetch_matches("upcoming")
print(upcoming_matches)
for m in upcoming_matches:
    insert_match(m)

print("Fetching Recent Matches...")
recent_matches = fetch_matches("recent")
print(recent_matches)
for m in recent_matches:
    insert_match(m)

print("Fetching Live Matches...")
live_matches = fetch_matches("live")
print(live_matches)
for m in live_matches:
    insert_match(m)


Fetching Upcoming Matches...
[{'matchInfo': {'matchId': 133077, 'seriesId': 10790, 'seriesName': 'Eastern Europe Cup 2025', 'matchDesc': '1st Match', 'matchFormat': 'T20', 'startDate': '1756450800000', 'endDate': '1756463400000', 'state': 'Upcoming', 'status': 'Match starts at Aug 29, 07:00 GMT', 'team1': {'teamId': 550, 'teamName': 'Hungary', 'teamSName': 'HUN', 'imageId': 172602}, 'team2': {'teamId': 1067, 'teamName': 'Serbia', 'teamSName': 'SRB', 'imageId': 247205}, 'venueInfo': {'id': 1689, 'ground': 'GB Oval', 'city': 'Szodliget', 'timezone': '+02:00', 'latitude': '47.730150', 'longitude': '19.146060'}, 'isTournament': True, 'seriesStartDt': '1756339200000', 'seriesEndDt': '1756684800000', 'isTimeAnnounced': True, 'stateTitle': 'Upcoming Match'}}, {'matchInfo': {'matchId': 133083, 'seriesId': 10790, 'seriesName': 'Eastern Europe Cup 2025', 'matchDesc': '2nd Match', 'matchFormat': 'T20', 'startDate': '1756465200000', 'endDate': '1756477800000', 'state': 'Upcoming', 'status': 'Match

# Innings

In [157]:
import requests

BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"
API_KEY = "d86ff93035msh5d9cc5b9a4e3a07p1ea55ejsnfb047b8ba175"

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}
def insert_innings(innings):
    cursor.execute("""
        INSERT INTO innings (
            match_id, innings_id, bat_team_id, bat_team_name, 
            bowl_team_id, bowl_team_name, runs, wickets, overs, run_rate, extras, time_score
        ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        ON DUPLICATE KEY UPDATE 
            runs=VALUES(runs), wickets=VALUES(wickets), overs=VALUES(overs), run_rate=VALUES(run_rate);
    """, (
        innings.get("matchId"),
        innings.get("inningsId"),
        innings.get("batTeamDetails", {}).get("batTeamId"),
        innings.get("batTeamDetails", {}).get("batTeamName"),
        innings.get("bowlTeamDetails", {}).get("bowlTeamId"),
        innings.get("bowlTeamDetails", {}).get("bowlTeamName"),
        innings.get("scoreDetails", {}).get("runs"),
        innings.get("scoreDetails", {}).get("wickets"),
        innings.get("scoreDetails", {}).get("overs"),
        innings.get("scoreDetails", {}).get("runRate"),
        innings.get("extrasData", {}).get("total"),
        innings.get("timeScore")
    ))
    print(f"✅ Innings inserted for Match {innings.get('matchId')} - Innings {innings.get('inningsId')}")

# ---------------------- Match Innings ----------------------

        
def fetch_match_innings():
    url = "https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/40381/hscard"
    response = requests.get(url, headers=headers)
    data = response.json()
    return data.get("scoreCard", [])   

innings_data = fetch_match_innings() # from your JSON
print("fetching innings")
print(innings_data)
for inn in innings_data:
    insert_innings(inn)

conn.commit()



fetching innings
[{'matchId': 40381, 'inningsId': 1, 'timeScore': 1640309044000, 'batTeamDetails': {'batTeamId': 27, 'batTeamName': 'Ireland', 'batTeamShortName': 'IRE', 'batsmenData': {'bat_1': {'batId': 1114, 'batName': 'Stirling', 'batShortName': '', 'isCaptain': False, 'isKeeper': False, 'runs': 5, 'balls': 5, 'dots': 0, 'fours': 1, 'sixes': 0, 'mins': 0, 'strikeRate': 100.0, 'outDesc': 'lbw b Nisarg Patel', 'bowlerId': 15475, 'fielderId1': 0, 'fielderId2': 0, 'fielderId3': 0, 'ones': 0, 'twos': 0, 'threes': 0, 'fives': 0, 'boundaries': 1, 'sixers': 0, 'wicketCode': 'LBW', 'isOverseas': False, 'inMatchChange': '', 'playingXIChange': ''}, 'bat_2': {'batId': 6710, 'batName': 'Balbirnie', 'batShortName': '', 'isCaptain': True, 'isKeeper': False, 'runs': 10, 'balls': 5, 'dots': 0, 'fours': 2, 'sixes': 0, 'mins': 0, 'strikeRate': 200.0, 'outDesc': 'c Sushant Modani b Netravalkar', 'bowlerId': 9748, 'fielderId1': 20217, 'fielderId2': 0, 'fielderId3': 0, 'ones': 0, 'twos': 0, 'threes': 0,

# Players

In [ ]:
import requests

BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"
API_KEY = "d86ff93035msh5d9cc5b9a4e3a07p1ea55ejsnfb047b8ba175"

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}


def insert_player(player):
    cursor.execute("""
        INSERT INTO players (player_id, name, role, batting_style, bowling_style)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        player.get("id"),   
        player.get("name"),
        player.get("role"),
        player.get("battingStyle","N/A"),
        player.get("bowlingStyle","N/A")
    ))
    print(f"✅ Player Inserted: {player.get('name')}")


    conn.commit()


def fetch_player():
    url = "https://cricbuzz-cricket.p.rapidapi.com/series/v1/3718/squads/15826"
    response = requests.get(url, headers=headers)
    data = response.json()
    return data  

print("Fetching Players...")
player_data = fetch_player()
print(player_data)
players_data = []

for p in player_data.get("player", []):
    if p.get("isHeader"):  # skip headers like "Wicketkeepers", "Batsmen"
        continue  

    players_data.append(p)
    insert_player(p)
    print("✅ Data Insering")

Fetching Players...


KeyboardInterrupt: 

# ---------------------- Scorecards ----------------------

In [ ]:
import requests

BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"
API_KEY = "d86ff93035msh5d9cc5b9a4e3a07p1ea55ejsnfb047b8ba175"

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}



# Ensure player exists in players table
def ensure_player(player_id, player_name):
    cursor.execute("""
        INSERT INTO players (player_id, name)
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE name = VALUES(name);
    """, (player_id, player_name))
    conn.commit()


#  Insert Batting Scorecard (per match)

def insert_batting_scorecard(match_id, innings_id, bat_info):
    # make sure player exists first
    ensure_player(bat_info.get("batId"), bat_info.get("batName"))

    cursor.execute("""
        INSERT INTO batting_scorecard 
        (match_id, innings_no, player_id, player_name, runs, balls, fours, sixes, strike_rate, dismissal)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """, (
        match_id,
        innings_id,
        bat_info.get("batId"),
        bat_info.get("batName"),
        bat_info.get("runs",0),
        bat_info.get("balls",0),
        bat_info.get("fours",0),
        bat_info.get("sixes",0),
        bat_info.get("strikeRate","0.0"),
        bat_info.get("outDesc")
    ))
    print(f"✅ Batting data inserted for {bat_info.get('batName')}")
    conn.commit()

#  Insert Bowling Scorecard (per match)

def insert_bowling_scorecard(match_id, innings_id, bowl_info):
    # make sure player exists first
    ensure_player(bowl_info.get("bowlerId"), bowl_info.get("bowlerName"))

    cursor.execute("""
        INSERT INTO bowling_scorecard
        (match_id, innings_id, bowler_id, bowler_name, overs, maidens, runs, wickets, economy)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """, (
        match_id,
        innings_id,
        bowl_info.get("bowlerId"),
        bowl_info.get("bowlerName"),
        bowl_info.get("overs"),
        bowl_info.get("maidens"),
        bowl_info.get("runs",0),
        bowl_info.get("wickets",0),
        bowl_info.get("economy")
    ))
    print(f"✅ Bowling data inserted for {bowl_info.get('bowlerName')}")
    conn.commit()


# ---------------------- Scorecards ----------------------
def fetch_scorecard():
    url = "https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/40381/hscard"
    response = requests.get(url, headers=headers)
    data = response.json()
    return data.get("scoreCard", [])


print("Fetching Scorecards...")
score_cards = fetch_scorecard()

for card in score_cards:
    match_id = card.get("matchId")
    innings_id = card.get("inningsId")
    
    # ---------- Batting Team ----------
    bat_team = card.get("batTeamDetails", {})
    batsmen_data = bat_team.get("batsmenData", {})
    
    print(batsmen_data)
    
    for bat_info in batsmen_data.values():  # Use .values() to get the actual batsman dicts
        insert_batting_scorecard(match_id, innings_id, bat_info)
        print(f"Inserting batting details for {bat_info.get('batName')}")
        
    # ---------- Bowling Team ----------
    bowl_team = card.get("bowlTeamDetails", {})
    bowlers_data = bowl_team.get("bowlersData", {})
    
    # print(bowlers_data)
    
    for bowl_info in bowlers_data.values():  # Use .values() to get the actual bowler dicts
        insert_bowling_scorecard(match_id, innings_id, bowl_info)
        print(f"Inserting bowling details for {bowl_info.get('bowlerName')}")


Fetching Scorecards...
{'bat_1': {'batId': 1114, 'batName': 'Stirling', 'batShortName': '', 'isCaptain': False, 'isKeeper': False, 'runs': 5, 'balls': 5, 'dots': 0, 'fours': 1, 'sixes': 0, 'mins': 0, 'strikeRate': 100.0, 'outDesc': 'lbw b Nisarg Patel', 'bowlerId': 15475, 'fielderId1': 0, 'fielderId2': 0, 'fielderId3': 0, 'ones': 0, 'twos': 0, 'threes': 0, 'fives': 0, 'boundaries': 1, 'sixers': 0, 'wicketCode': 'LBW', 'isOverseas': False, 'inMatchChange': '', 'playingXIChange': ''}, 'bat_2': {'batId': 6710, 'batName': 'Balbirnie', 'batShortName': '', 'isCaptain': True, 'isKeeper': False, 'runs': 10, 'balls': 5, 'dots': 0, 'fours': 2, 'sixes': 0, 'mins': 0, 'strikeRate': 200.0, 'outDesc': 'c Sushant Modani b Netravalkar', 'bowlerId': 9748, 'fielderId1': 20217, 'fielderId2': 0, 'fielderId3': 0, 'ones': 0, 'twos': 0, 'threes': 0, 'fives': 0, 'boundaries': 2, 'sixers': 0, 'wicketCode': 'CAUGHT', 'isOverseas': False, 'inMatchChange': '', 'playingXIChange': ''}, 'bat_3': {'batId': 11131, 'ba

# ---------------------- Player Career Stats ----------------------



In [ ]:
import requests

BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"
API_KEY = "9d19b813acmsh770d33e52e3a171p11379djsn7dd0eed30400"

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}



#  Insert Batting Career Stats
def insert_stats(stats):
    cursor.execute("""
        INSERT INTO batting_stats (player_id, bat, bowl, name, nickName, height, role, birthPlace, intlTeam, teams)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """, (
        stats.get("id"),   
        stats.get("bat"),
        stats.get("bowl"),
        stats.get("name"),
        stats.get("nickName"),
        stats.get("height"),
        stats.get("role"),
        stats.get("birthPlace"),
        stats.get("intlTeam"),
        stats.get("teams")
    ))

    print(f"✅ Inserted Player {stats.get('name')}")
    conn.commit()


  
# ---------------------- Player Career Stats ----------------------
def fetch_player_stats(p):
    url = f"{BASE_URL}/stats/v1/player/{p}"
    response = requests.get(url, headers=headers)
    return response.json()

# ---------------------- Run ----------------------
player_ids = ["8733","6635"]  # example IDs

print("Fetching Player Career Stats...")
for p in player_ids:
    stats = fetch_player_stats(p)
    print(stats)
    insert_stats(stats)
    # insert batting career if available
    

Fetching Player Career Stats...
{'id': '8733', 'bat': 'Right Handed Bat', 'name': 'KL Rahul', 'nickName': 'Rahul', 'role': 'WK-Batsman', 'birthPlace': 'Mangaluru, Karnataka ', 'intlTeam': 'India', 'teams': 'India, Karnataka, East Zone, South Zone, Royal Challengers Bengaluru, India A, Sunrisers Hyderabad, Indians, Hubli Tigers, Indian Board Presidents XI, Bellary Tuskers, Punjab Kings, Asia XI, Lucknow Super Giants, Delhi Capitals', 'DoB': 'April 18, 1992 (33 years)', 'image': 'http://i.cricketcb.com/stats/img/faceImages/8733.jpg', 'bio': " <br/> Hailing from a background of high academic standards, KL Rahul was born and brought up in the prestigious National Institute of Technology Karnataka, where his father is a professor in the department of Civil Engineering. He finished his schooling in NITK and grew to develop keen interest in cricket. Having started off with batting on the grass-less field of the college, Rahul's parents spotted a spark in him and decided that he must receive f

# ----------------------  Players career details----------------------

In [35]:
import requests

BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"
API_KEY = "9d19b813acmsh770d33e52e3a171p11379djsn7dd0eed30400"

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

# ---------------------- Insert Func ----------------------
def insert_matches_players(data):
    cursor.execute("""
        INSERT INTO match_players
            (match_id, player_id, format, team_name, debut, last_played, debut_match_id, last_match_id)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
    """, (
        data.get("match_id"),
        data.get("player_id"),
        data.get("format"),
        data.get("team_name"),
        data.get("debut"),
        data.get("last_played"),
        data.get("debut_match_id"),
        data.get("last_match_id")
    ))
    conn.commit()
    print(f"✅ Inserted {data.get('format')} career record for Player {data.get('player_id')}")


# ---------------------- Fetch Career ----------------------
def fetch_match_players(pd):
    url = f"{BASE_URL}/stats/v1/player/{pd}/career"
    response = requests.get(url, headers=headers)
    data = response.json()
    return data.get("values", [])   # this is a list of formats


# ---------------------- Main Loop ----------------------
player_ids = ["8733", "6635"]

for pd in player_ids:
    career_data = fetch_match_players(pd)
    print(career_data)

    for record in career_data:
        insert_data = {
            "match_id": None,  # keep NULL since this isn’t linked to a single match
            "player_id": pd,
            "format": record.get("name"),
            "team_name": record.get("team", "N/A"),  # fallback if missing
            "debut": record.get("debut"),
            "last_played": record.get("lastPlayed"),
            "debut_match_id": record.get("debutMatchId"),
            "last_match_id": record.get("lastMatchId")
        }
        insert_matches_players(insert_data)




    

[{'name': 't20', 'debut': 'vs Zimbabwe,  2016-06-18, Harare Sports Club', 'lastPlayed': 'vs England,  2022-11-10, Adelaide Oval', 'debutMatchID': '16610', 'lastPlayedMatchId': '43201'}, {'name': 'test', 'debut': 'vs Australia,  2014-12-26, Melbourne Cricket Ground', 'lastPlayed': 'vs England,  2025-07-31, Kennington Oval', 'debutMatchID': '13842', 'lastPlayedMatchId': '105780'}, {'name': 'odi', 'debut': 'vs Zimbabwe,  2016-06-11, Harare Sports Club', 'lastPlayed': 'vs New Zealand,  2025-03-09, Dubai International Cricket Stadium', 'debutMatchID': '16607', 'lastPlayedMatchId': '112469'}, {'name': 'ipl', 'debut': 'vs Kolkata Knight Riders,  2013-04-11, M.Chinnaswamy Stadium', 'lastPlayed': 'vs Punjab Kings,  2025-05-24, Sawai Mansingh Stadium', 'debutMatchID': '11876', 'lastPlayedMatchId': '118874'}, {'name': 'cl', 'debut': 'Not played', 'lastPlayed': 'Not played'}]
✅ Inserted t20 career record for Player 8733
✅ Inserted test career record for Player 8733
✅ Inserted odi career record for

# -----------------Venues-----------------

In [ ]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/venues/v1/45"

headers = {
	"x-rapidapi-key": "221c95f933msh79fdb3adf8f84e5p102f11jsn2da1f5a3f788",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())